#Setup and Architecture

Here we will find the following sections: 
   1. Import Section
   2. Defining the training/accuracy functions 
   3. Dataset Generation
   4. Training Stage 1 AlexNet model

#FORMATTING NOTES:

1. **Input size for images is:** 224 by 224
2. All images should be in greyscale (With channel dimension included)

In [0]:
#Import Section

import numpy as np
np.set_printoptions(suppress=True)
import scipy.signal as sg
from PIL import Image, ImageDraw
import requests
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt # for plotting
import torch.optim as optim #for gradient descent
import torchvision
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, utils, models

import csv
import seaborn as sn
import math
import os
import shutil
import random
import cv2
from scipy import ndimage, misc
import itertools
from skimage import io, transform

#20 Samples Import


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# define training and test data directories
#THIS IS DIFFERENT FOR EVERYONE!!!!
#  - Emily: '/content/drive/My Drive/20 samples'
#  - Hiranya: '/content/drive/My Drive/APS360 2019 September Team/Steel Defect Detection/20 samples'
train_dir = '/content/drive/My Drive/APS360 2019 September Team/Steel Defect Detection/20 samples'


data_transform = transforms.Compose([transforms.Resize((800,128)),transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
train_data = datasets.ImageFolder(train_dir, transform=data_transform)



In [0]:
print (train_data)

Dataset ImageFolder
    Number of datapoints: 22
    Root location: /content/drive/My Drive/APS360 2019 September Team/Steel Defect Detection/20 samples
    StandardTransform
Transform: Compose(
               Resize(size=(800, 128), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )


#Define Training and Accuracy Functions

Baseline model training code

In [0]:
def get_accuracy(model, data_loader, useGPU=True):
    correct = 0
    total = 0
    for imgs, labels in data_loader:
        #To Enable GPU Usage
        if useGPU and torch.cuda.is_available():
            imgs = imgs.cuda()
            labels = labels.cuda()
            model = model.cuda()
        output = model(imgs)
        
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
        
    return correct / total

In [0]:
def train(model, train_data, val_data, batch_size=64, num_epochs=1, learning_rate=0.01, useGPU=True):
    
    #Put data in data loaders
    train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=0, shuffle=False)
    val_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=0, shuffle=False)
   
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), learning_rate)
   
    iters, losses, train_acc, val_acc = [], [], [], []

    if useGPU and torch.cuda.is_available():
        model = model.cuda()
        print("Training on GPU")
   
    # training
    n = 0 # the number of iterations
    for epoch in range(num_epochs):
        for imgs, labels in iter(train_data_loader):
          #To Enable GPU Usage
          if useGPU and torch.cuda.is_available():
              imgs = imgs.cuda()
              labels = labels.cuda()
          
          out = model(imgs)             # forward pass

          loss = criterion(out, labels) # compute the total loss
          loss.backward()               # backward pass (compute parameter updates)
          optimizer.step()              # make the updates for each parameter
          optimizer.zero_grad()         # a clean up step for PyTorch

          # save the current training information
          iters.append(n)
          losses.append(float(loss)/batch_size)             # compute *average* loss
          train_acc.append(get_accuracy(model, train_data_loader, useGPU)) # compute training accuracy 
          val_acc.append(get_accuracy(model, val_data_loader, useGPU))  # compute validation accuracy
          n += 1
        print(("Epoch {}: Train loss: {}, "+"Train accuracy: {}").format(epoch + 1,losses[epoch],train_acc[epoch]))
            
        # Save the current model (checkpoint) to a file
        model_path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(model.name,
                                                    batch_size,
                                                    str(learning_rate).replace('.', '-'),
                                                    epoch)
        torch.save(model.state_dict(), model_path + ".pth")         

    # plotting
    plt.title("Training Curve")
    plt.plot(iters, losses, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(iters, train_acc, label="Train")
    plt.plot(iters, val_acc, label="Validation")
    plt.xlabel("Iterations")
    plt.ylabel("Training Accuracy")
    plt.legend(loc='best')
    plt.show()

    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))

# Split train Data into 1 and 0

In [0]:
#Link to competition: https://www.kaggle.com/c/severstal-steel-defect-detection/overview
#Upload the Kaggle JSON file (download it from the drive to your computer first)
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"farhanwadia","key":"2c92598698ee94f469567eb3af100ca4"}'}

In [0]:
#Mount the kaggle.json file
!ls -lha kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 67 Nov 21 02:56 kaggle.json


In [0]:
#Extract required files
!kaggle competitions download -c severstal-steel-defect-detection -q
!unzip -q train.csv.zip
!unzip -q train_images.zip -d train_images

In [0]:
#Load the dataset and list of filenames

def removeDuplicates(lst):
    #Remove duplicate values from a list
    return list(dict.fromkeys(lst))

#Returns the run-length encoded pixels for a filename and given classID from 1 to 4
def getEncodedPixels(filename, classID):
    return data[filenameAndClassIndex[filename + "_" + str(classID)]][1]

#Load train.csv as a list
trainingDataPath = "/content/train.csv"
with open(trainingDataPath, 'r') as file:
    data = list(csv.reader(file, delimiter=","))

filenameAndClassIndex = {} #dictionary to index data array based on first column's value (ImageId_ClassId)   
filenames = []

for i in range(1, len(data)):
    filenameAndClassIndex[data[i][0]] = i
    filenames.append(data[i][0].split("_")[0])
filenames = removeDuplicates(filenames) # remove duplicates from the list of filenames

In [0]:
def getOneClassFilenames(classID, filenamesByClass):
    #gets a list of filenames that are only in one class and not in multiple classes
    for i in range(5):
        others = set()
        if i != classID:
            #Get a union of all the image names not in the class
            others = others.union(set(filenamesByClass[i]))
    return list(set(filenamesByClass[classID]).difference(others)) #only keep the filenames not in others
def getImage(filename):
    #returns a PIL Image object of the image with name 'filename'
    os.chdir("/content/train_images")
    picture = Image.open(filename)
    return picture

In [0]:
#Get the number of images with defects in each class

#numInClassCounter[0] is the number of images w/ no defects, numInClassCounter[1] is the number of images w/ class 1 defects... 
#filenamesByClass is a 2D list. The first dimension corresponds to the classID (0: No defects, ..., 1: Class 1, 4: Class 4)

numClasses = 5
numInClassCounter = [0, 0, 0, 0, 0]
filenamesByClass = [[], [], [], [], []]
for img_file in filenames:
    noDefects = True
    for classID in range(1,5):
        if getEncodedPixels(img_file, classID) != '': 
              numInClassCounter[classID] += 1 
              filenamesByClass[classID].append(img_file)
              noDefects = False
    if noDefects:
        numInClassCounter[0] +=1
        filenamesByClass[0].append(img_file)

In [0]:
#Get a list of filenames which only have 1 class of defect. First dimension is the class, second dimension is the list.
#These files will be used to create the histogram
uniqueFilenamesByClass = []
for i in range(5):
    uniqueFilenamesByClass.append(getOneClassFilenames(i, filenamesByClass))

In [0]:
os.makedirs('content/split/0')
changeDir= 'content/split/0/'
trainDir='/content/train_images/'
for i in uniqueFilenamesByClass[0]:
  imgDir=trainDir+i
  imgchangeDir=changeDir + i
  os.rename(imgDir, imgchangeDir)

In [0]:
from pathlib import Path

In [0]:
os.makedirs('content/split/1')
changeDir= 'content/split/1/'
trainDir='/content/train_images/'
for j in range(1,5):
  for i in uniqueFilenamesByClass[j]:
    imgDir=trainDir+i
    imgchangeDir=changeDir + i
    my_file = Path(imgDir)
    if my_file.is_file():
      os.rename(imgDir, imgchangeDir)

In [0]:
# define training and test data directories
train_dir = 'content/split/'

# load and transform data using ImageFolder
data_transform = transforms.Compose([transforms.Resize((800,128)),transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_data = datasets.ImageFolder(train_dir, transform=data_transform)

In [0]:
# Include all import statements required over here
import csv
import numpy as np
np.set_printoptions(suppress=True)
import matplotlib.pyplot as plt
import seaborn as sn
from PIL import Image, ImageDraw
import math
import os
import shutil
import random
import cv2
from scipy import ndimage, misc
import itertools
from skimage import io, transform
from sklearn import svm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, transforms, utils, models

In [0]:
print(np.shape(train_data.targets))

(12568,)


In [0]:
BaselineSVM = svm.NuSVC(gamma = 'scale')

train_loader = torch.utils.data.DataLoader(train_data, batch_size = 64, shuffle = True)
counter = 0
accuracy = []

for imgs, labels in iter(train_loader):
  floatLabels =  labels.float() 
  #print(imgs)
  #print(floatLabels)
  imgs = imgs.view(-1, 800 * 128 *3)

  BaselineSVM.fit(imgs, floatLabels)
  accuracy.append(BaselineSVM.score(imgs, floatLabels)) 
  print('Accuracy of binary baseline SVM for batch: ', counter,' is: ', accuracy[counter])

  counter = counter +1

#np.random.shuffle(training_images)
#np.random.shuffle(training_Targets)

#print(training_images_locations)
#print(training_Targets)

total = sum(accuracy)
averageAcc = total/counter
print('Average accuracy per batch is: ', averageAcc)

Accuracy of binary baseline SVM for batch:  0  is:  0.9375
Accuracy of binary baseline SVM for batch:  1  is:  0.953125
Accuracy of binary baseline SVM for batch:  2  is:  0.90625
Accuracy of binary baseline SVM for batch:  3  is:  0.96875
Accuracy of binary baseline SVM for batch:  4  is:  0.9375
Accuracy of binary baseline SVM for batch:  5  is:  0.9375
Accuracy of binary baseline SVM for batch:  6  is:  0.984375
Accuracy of binary baseline SVM for batch:  7  is:  0.921875
Accuracy of binary baseline SVM for batch:  8  is:  0.96875
Accuracy of binary baseline SVM for batch:  9  is:  0.96875
Accuracy of binary baseline SVM for batch:  10  is:  0.9375
Accuracy of binary baseline SVM for batch:  11  is:  0.96875
Accuracy of binary baseline SVM for batch:  12  is:  0.96875
Accuracy of binary baseline SVM for batch:  13  is:  0.9375
Accuracy of binary baseline SVM for batch:  14  is:  0.9375
Accuracy of binary baseline SVM for batch:  15  is:  0.96875
Accuracy of binary baseline SVM for b